In [8]:
# install api interface at https://github.com/cwendt94/ff-espn-api/

import requests

# get hardcoded cookies from browser


# helper function from ff-espn-api
def checkRequestStatus(status: int) -> None:
    if 500 <= status <= 503:
            raise Exception(status)
    if status == 401:
        raise Exception("Access Denied")

    elif status == 404:
        raise Exception("Invalid League")

    elif status != 200:
        raise Exception('Unknown %s Error' % status)


year = 2020
league_id = 83174673


# fetch league. need some data from cookies to fill in here
swid = '{9F7C3455-C43D-42A6-9D1A-AEB707CB5F0B}'
espn_s2 = 'AEAy3mkAv%2FsnK2YbgEVRFKpOw6%2ByQVeXH5BDBOlzgAiBg646ZtRuneInbY1z9QYrt1Ws8r6Cc8pS0g%2BVi7HmU0b%2F0lxgkP9csOvk9S01pusHVMJOZ9GfMZblZ01U9NjD46R2N%2BI21guim1Lga4TfVKqE6rCLMJvQszv5ZKdFgjngR98gxiClp9R3ZxzJ7cTLJufuir4JGnzKZ9RwOCL1%2BOaFy8qFEGwEVNGkuMY4Hj4zSR%2BL9a5fatMYEWNXvk4muhbgckHixSZiqvSnL6pyniu98SRFEpigMIkWgSJ%2FTfNvCg%3D%3D'
cookies = {
    'espn_s2': espn_s2,
    'SWID': swid
}
ENDPOINT = "https://fantasy.espn.com/apis/v3/games/FFL/seasons/" + str(year) + "/segments/0/leagues/" + str(league_id)
params = ''

# test request
r = requests.get(ENDPOINT, params=params, cookies=cookies)
status = r.status_code
checkRequestStatus(status)
data = r.json()
print(data)

{'gameId': 1, 'id': 83174673, 'members': [{'displayName': 'espn04014086', 'id': '{23752840-5024-4BC3-B528-4050243BC3EE}', 'isLeagueManager': False}, {'displayName': 'YungFly', 'id': '{2A52739B-A4D2-440F-B78D-C1282294B36E}', 'isLeagueManager': False}, {'displayName': 'Ball, call, install', 'id': '{5E1655AA-B7E8-4112-8208-4024E2B806B4}', 'isLeagueManager': False}, {'displayName': 'espn57414091', 'id': '{86F13096-2F1D-44BC-B130-962F1DA4BCEA}', 'isLeagueManager': False}, {'displayName': 'broncoz16', 'id': '{89BABC79-3E89-4484-B7E0-91FE30264A36}', 'isLeagueManager': False}, {'displayName': 'typhe12', 'id': '{8BCE107A-C396-4216-A165-204DCBB7ACED}', 'isLeagueManager': False}, {'displayName': 'ESPNFAN6778113520', 'id': '{9DAF6913-3970-416F-BECB-D0DE77788ADB}', 'isLeagueManager': False}, {'displayName': 'shottysnipesonthepit', 'id': '{9F7C3455-C43D-42A6-9D1A-AEB707CB5F0B}', 'isLeagueManager': False}, {'displayName': 'JackMichaels55', 'id': '{AED6F6FD-B3C1-49EC-9329-C5E971023ED7}', 'isLeagueMana

In [15]:
# fetch players, get general data about players, don't really remember what's in here
params = {
    'scoringPeriodId': 0,
    'view': 'players_wl',
}
endpoint = 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/' + str(year) + '/players'
r = requests.get(endpoint, params=params, cookies=cookies)
status = r.status_code
checkRequestStatus(status)
data = r.json()
print(data)

[{'defaultPositionId': 2, 'droppable': True, 'eligibleSlots': [2, 3, 23, 7, 20, 21], 'firstName': 'Ameer', 'fullName': 'Ameer Abdullah', 'id': 2576336, 'lastName': 'Abdullah', 'ownership': {'percentOwned': 0.07516603567826986}, 'proTeamId': 16, 'universeId': 2}, {'defaultPositionId': 13, 'droppable': True, 'eligibleSlots': [13, 14, 15, 20, 21], 'firstName': 'Johnathan', 'fullName': 'Johnathan Abram', 'id': 3728305, 'lastName': 'Abram', 'ownership': {'percentOwned': 11.163093687078053}, 'proTeamId': 13, 'universeId': 1}, {'defaultPositionId': 12, 'droppable': True, 'eligibleSlots': [12, 14, 15, 20, 21], 'firstName': 'Delrick', 'fullName': 'Delrick Abrams', 'id': 4360643, 'lastName': 'Abrams', 'ownership': {'percentOwned': 0.0}, 'proTeamId': 1, 'universeId': 2}, {'defaultPositionId': 11, 'droppable': True, 'eligibleSlots': [10, 15, 20, 21], 'firstName': 'Sam', 'fullName': 'Sam Acho', 'id': 14152, 'lastName': 'Acho', 'ownership': {'percentOwned': 0.008595003438001375}, 'proTeamId': 0, 'un

In [13]:
import numpy as np

# fetch players, get scoring data
params = {
    'scoringPeriodId': 1
}
endpoint = 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/' + str(year) + '/segments/0/leagues/' + str(league_id) + '?view=mMatchup&view=mMatchupScore'
r = requests.get(endpoint, params=params, cookies=cookies)
status = r.status_code
checkRequestStatus(status)
d = r.json()

# get player scoring data for all weeks
import pandas as pd

slotcodes = {
    0 : 'QB', 2 : 'RB', 4 : 'WR',
    6 : 'TE', 16: 'Def', 17: 'K',
    20: 'Bench', 21: 'IR', 23: 'Flex'
}

# holder variable
data = []

# iterate teams
for tm in d['teams']:
    tmid = tm['id']

    # iterate players on teams
    for p in tm['roster']['entries']:
        name = p['playerPoolEntry']['player']['fullName']
        slot = p['lineupSlotId']
        pos  = slotcodes[slot]

        # injured status (need try/exc bc of D/ST)
        inj = 'NA'
        try:
            inj = p['playerPoolEntry']['player']['injuryStatus']
        except:
            pass

        # grab projected/actual points
        proj, act = None, None
        for stat in p['playerPoolEntry']['player']['stats']:
            if stat['scoringPeriodId'] != week:
                continue
            if stat['statSourceId'] == 0:
                act = stat['appliedTotal']
            elif stat['statSourceId'] == 1:
                proj = stat['appliedTotal']

        data.append([
            week, tmid, name, slot, pos, inj, proj, act
        ])
print('\nComplete.')

data = pd.DataFrame(data, 
                columns=['Week', 'Team', 'Player', 'Slot', 
                             'Pos', 'Status', 'Proj', 'Actual'])




Complete.


In [14]:
data

,Week,Team,Player,Slot,Pos,Status,Proj,Actual
0,1,1,Clyde Edwards-Helaire,2,RB,QUESTIONABLE,15.254842,19.8
1,1,1,Aaron Jones,2,RB,ACTIVE,15.269672,17.6
2,1,1,DJ Moore,4,WR,OUT,15.028489,9.4
3,1,1,Cooper Kupp,4,WR,ACTIVE,14.116474,8.0
4,1,1,A.J. Brown,4,WR,ACTIVE,13.662708,8.9
...,...,...,...,...,...,...,...,...
145,1,10,Bills D/ST,16,Def,NA,9.390461,11.0
146,1,10,Zack Moss,23,Flex,ACTIVE,9.853133,11.7
147,1,10,Breshad Perriman,20,Bench,ACTIVE,8.436784,4.7
148,1,10,Robbie Gould,17,K,ACTIVE,7.922809,10.0
